In [13]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [14]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

In [15]:
df

,Open,High,Low,Close,Volume,signal
timestamp,,,,,,
2022-01-01 04:00:00,172.790,173.130,172.480,172.930,49675,SignalNone
2022-01-01 05:00:00,172.940,173.110,171.470,171.470,68973,SignalNone
2022-01-01 06:00:00,171.490,174.310,171.340,173.220,95595,SignalNone
2022-01-01 07:00:00,173.220,173.740,172.260,172.470,95494,SignalNone
2022-01-01 08:00:00,172.450,173.440,172.140,173.160,57926,SignalNone
...,...,...,...,...,...,...
2023-12-31 02:00:00,102.718,103.063,101.208,101.411,731765,SignalNone
2023-12-31 03:00:00,101.413,101.850,100.044,100.738,970135,SignalLong
2023-12-31 04:00:00,100.734,100.939,99.635,100.743,858035,SignalNone


In [3]:
data = vbt.Data.from_data(df)
data

In [ ]:
# data.plot().show()

In [17]:
open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

In [6]:
# vbt.phelp(vbt.MA.run)

In [18]:
adx = vbt.ADX.run(high_price, low_price, close_price, window=14)
atr = vbt.ATR.run(high_price, low_price, close_price, window=14)
bbands = vbt.BBANDS.run(close_price, window=14)
rsi = vbt.RSI.run(close_price)
sma = vbt.MA.run(close_price, window=20)
strend = vbt.SUPERTREND.run(high_price, low_price, close_price, period=7, multiplier=3)

data.data['symbol']['ADX'] = adx.adx
data.data['symbol']['ATR'] = atr.atr
data.data['symbol']['BBAND'] = bbands.bandwidth
data.data['symbol']['RSI'] = rsi.rsi
data.data['symbol']['SMA'] = sma.ma
data.data['symbol']['STREND'] = strend.trend
data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = ['Close', 'ADX', 'ATR', 'BBAND', 'RSI', 'SMA', 'STREND']

X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

In [9]:
from sklearn.model_selection import train_test_split
split_percentage = 0.7
split = int(split_percentage*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split_percentage)


In [10]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=5, random_state=13).fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [11]:
y_pred_proba = clf.predict_proba(X_test)

In [ ]:
y_pred_proba

In [147]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score
# report = classification_report(y_test, y_pred)
# accuracy = accuracy_score(y_test, y_pred)
# print(accuracy)

0.9204502098435712


In [148]:
# Store the test dataset in new dataframe df_split
df_split = data.data['symbol'][split:].copy()

# Store the decision tree's predicted output to signal column of df_split dataframe
df_split.loc[:, "signal"] = y_pred

# df_split

In [149]:
signal = df_split['signal']
entries = signal == 'SignalLong'
exits = signal == 'SignalShort'


In [150]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [151]:
# import joblib
# def save_model(model, model_filename='trained_model.joblib'):
#     joblib.dump(model, model_filename)
#     print(f'Model saved as {model_filename}')
# save_model(clf)

In [152]:
pf.plot(settings=dict(bm_returns=False)).show()

In [153]:
pf.stats()

Start                         2023-05-26 21:00:00+00:00
End                           2023-12-31 06:00:00+00:00
Period                                218 days 10:00:00
Start Value                                       100.0
Min Value                                     99.887214
Max Value                                    253.018427
End Value                                    232.796495
Total Return [%]                             132.796495
Benchmark Return [%]                         421.028013
Total Time Exposure [%]                       47.520031
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              20.299032
Max Drawdown Duration                 110 days 03:00:00
Total Orders                                         45
Total Fees Paid                                     0.0
Total Trades                                         23
Win Rate [%]                                  63.636364
Best Trade [%]                                54